## 等级

最高等级为49级。10级之内通过经验值升级。

到达以下等级（9-10，19-20，29-30，39-40）时，
需要达成特殊条件才能升级，升级后获得游戏阶段提升。

#### 加点（正常获得1点，每10级获得5点）

+ 体力，每点+4HP
+ 精神，每点+1SP
+ 攻击，每点增加物理和基础魔法攻击
+ 魔法，每点增加魔法效果
+ 运气，每点增加产生好结果的概率

> 洗点：允许重新洗本阶段的加点

In [212]:
import pandas as pd
from dataclasses import dataclass

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

initial_hp = 50
initial_sp = 20

hp_per_level = 4
sp_per_level = 1

max_hp = initial_hp
max_sp = initial_sp

data = []

for i in range(1, 59+1):
    rank = i // 10 + 1
    if i == 1:
        pass
    else:
        if i % 10 == 0:
            max_hp += 5 * hp_per_level
            max_sp += 5 * sp_per_level
        else:
            max_hp += hp_per_level
            max_sp += sp_per_level
    # data.append([rank, i, exp, max_hp, max_sp])
    data.append({
        'rank': rank,
        'level': i,
        'hp': max_hp,
        'sp': max_sp,
    })

df = pd.DataFrame(data)
df.set_index(['rank', 'level'], inplace=True)

# 计算加点全部加HP或者SP的最大值
df['hp_ratio'] = df['hp'] / initial_hp
df['sp_ratio'] = df['sp'] / initial_sp


In [213]:
from collections import defaultdict

@dataclass
class ExpFactor:
    rank: int
    base_factor: float      # rank基本时长倍率
    level_factor: float     # 每级比上一级多需要的升级时长倍率

exp_factors = [
    ExpFactor(1, 1.0, level_factor=1.050),
    ExpFactor(2, 2.0, level_factor=1.040),
    ExpFactor(3, 3.5, level_factor=1.030),
    ExpFactor(4, 5.0, level_factor=1.025),
    ExpFactor(5, 6.5, level_factor=1.025),
    ExpFactor(6, 8.0, level_factor=1.020),
]

exp_list = defaultdict(list[tuple[int, float]])

for factor in exp_factors:
    rank = factor.rank
    if rank == 1:
        assert factor.base_factor == 1
        exp_list[rank].append((1, 1.0))
        for lv in range(2, 10):
            _, prev_exp = exp_list[rank][-1]
            exp_list[rank].append((lv, prev_exp * factor.level_factor))
    else:
        # 求等比数列的初项
        # 总时长 = 初项 * (1 - q^n) / (1 - q)
        rank_1_exp_sum = sum([exp for _, exp in exp_list[1]])
        n = 10
        q = factor.level_factor
        S = rank_1_exp_sum * factor.base_factor
        first_exp = S * (1 - q) / (1 - q**n)
        exp_list[rank].append((0, first_exp))
        for lv in range(1, 10):
            _, prev_exp = exp_list[rank][-1]
            exp_list[rank].append((lv, prev_exp * q))

# put exp_list into df
for rank, exps in exp_list.items():
    for lv, exp in exps:
        lv = lv + (rank - 1) * 10
        df.loc[(rank, lv), 'exp'] = round(exp * 10000)

df['exp'] = df['exp'].astype(int)
# df['exp_diff'] = df['exp'].diff()
df['exp_total_rate'] = df['exp'] / df.iloc[0]['exp']
df['exp_pct_change'] = df['exp'].pct_change().map(lambda x: f'{x:.2%}')
df.head(1000)

hp  sp  hp_ratio  sp_ratio    exp  exp_total_rate exp_pct_change
rank level                                                                   
1    1       50  20      1.00      1.00  10000          1.0000           nan%
     2       54  21      1.08      1.05  10500          1.0500          5.00%
     3       58  22      1.16      1.10  11025          1.1025          5.00%
     4       62  23      1.24      1.15  11576          1.1576          5.00%
     5       66  24      1.32      1.20  12155          1.2155          5.00%
     6       70  25      1.40      1.25  12763          1.2763          5.00%
     7       74  26      1.48      1.30  13401          1.3401          5.00%
     8       78  27      1.56      1.35  14071          1.4071          5.00%
     9       82  28      1.64      1.40  14775          1.4775          5.00%
2    10     102  33      2.04      1.65  18368          1.8368         24.32%
     11     106  34      2.12      1.70  19103          1.9103          4.00%
     12     110  35      2.20      1.75  19867          1.9867          4.00%
     13     114  36      2.28      1.80  20662          2.0662          4.00%
     14     118  37      2.36      1.85  21488          2.1488          4.00%
     15     122  38      2.44      1.90  22348          2.2348          4.00%
     16     126  39      2.52      1.95  23242          2.3242          4.00%
     17     130  40      2.60      2.00  24171          2.4171          4.00%
     18     134  41      2.68      2.05  25138          2.5138          4.00%
     19     138  42      2.76      2.10  26144          2.6144          4.00%
3    20     158  47      3.16      2.35  33665          3.3665         28.77%
     21     162  48      3.24      2.40  34675          3.4675          3.00%
     22     166  49      3.32      2.45  35715          3.5715          3.00%
     23     170  50      3.40      2.50  36786          3.6786          3.00%
     24     174  51      3.48      2.55  37890          3.7890          3.00%
     25     178  52      3.56      2.60  39027          3.9027          3.00%
     26     182  53      3.64      2.65  40198          4.0198          3.00%
     27     186  54      3.72      2.70  41404          4.1404          3.00%
     28     190  55      3.80      2.75  42646          4.2646          3.00%
     29     194  56      3.88      2.80  43925          4.3925          3.00%
4    30     214  61      4.28      3.05  49211          4.9211         12.03%
     31     218  62      4.36      3.10  50441          5.0441          2.50%
     32     222  63      4.44      3.15  51702          5.1702          2.50%
     33     226  64      4.52      3.20  52995          5.2995          2.50%
     34     230  65      4.60      3.25  54320          5.4320          2.50%
     35     234  66      4.68      3.30  55678          5.5678          2.50%
     36     238  67      4.76      3.35  57070          5.7070          2.50%
     37     242  68      4.84      3.40  58496          5.8496          2.50%
     38     246  69      4.92      3.45  59959          5.9959          2.50%
     39     250  70      5.00      3.50  61458          6.1458          2.50%
5    40     270  75      5.40      3.75  63974          6.3974          4.09%
     41     274  76      5.48      3.80  65573          6.5573          2.50%
     42     278  77      5.56      3.85  67213          6.7213          2.50%
     43     282  78      5.64      3.90  68893          6.8893          2.50%
     44     286  79      5.72      3.95  70615          7.0615          2.50%
     45     290  80      5.80      4.00  72381          7.2381          2.50%
     46     294  81      5.88      4.05  74190          7.4190          2.50%
     47     298  82      5.96      4.10  76045          7.6045          2.50%
     48     302  83      6.04      4.15  77946          7.7946          2.50%
     49     306  84      6.12      4.20  79895          7.9895          2.50%
6    50     326  89      6.52      

In [214]:
rank_df = df.groupby('rank').agg({'exp': 'sum'})
rank_df['exp_diff_rate'] = rank_df['exp'].pct_change().map(lambda x: f'{x:.2%}')
rank_df['exp_total_rate'] = (rank_df['exp'] / rank_df.iloc[0]['exp'])
rank_df

,exp,exp_diff_rate,exp_total_rate
rank,,,
1,110266,nan%,1.000000
2,220531,100.00%,1.999991
3,385931,75.00%,3.500000
4,551330,42.86%,5.000000
5,716725,30.00%,6.499964
6,882124,23.08%,7.999964
